In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df_test = pd.read_csv('/Users/egmzvalerio/apps/iteso-data-analytics/data/raw/test.csv')
df_train = pd.read_csv('/Users/egmzvalerio/apps/iteso-data-analytics/data/raw/Titanic-Dataset.csv')

In [3]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


In [7]:
df_train['Sex'] = df_train['Sex'].replace({'female':0, 'male':1})
df_test['Sex'] = df_test['Sex'].replace({'female':0, 'male':1})

In [8]:
df_train['Embarked'] = df_train['Embarked'].replace({'Q': 0, 'S': 1, 'C': 2})
df_test['Embarked'] = df_test['Embarked'].replace({'Q': 0, 'S': 1, 'C': 2})

In [9]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].mean())

In [10]:
bins = [0, 8, 15, 18, 25, 40, 60, 100]
grupos = ['1', '2', '3', '4', '5', '6', '7']
df_train['Age'] = pd.cut(df_train['Age'], bins, labels = grupos)
df_test['Age'] = pd.cut(df_test['Age'], bins, labels = grupos)

In [11]:
df_train = df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
df_test = df_test.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

In [12]:
moda = df_train['Embarked'].mode()[0]
df_train['Embarked'] = df_train['Embarked'].fillna(moda)

moda2 = df_test['Embarked'].mode()[0]
df_test['Embarked'] = df_test['Embarked'].fillna(moda2)

In [13]:
moda3 = df_test['Fare'].mode()[0]
df_test['Fare'] = df_test['Fare'].fillna(moda2)

In [14]:
df_train.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [15]:
df_test.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [16]:
X = df_train.drop(columns = 'Survived', axis = 1)
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,4,1,0,7.2500,1.0
1,1,0,5,1,0,71.2833,2.0
2,3,0,5,0,0,7.9250,1.0
3,1,0,5,1,0,53.1000,1.0
4,3,1,5,0,0,8.0500,1.0
...,...,...,...,...,...,...,...
886,2,1,5,0,0,13.0000,1.0
887,1,0,4,0,0,30.0000,1.0
888,3,0,5,1,2,23.4500,1.0
889,1,1,5,0,0,30.0000,2.0


In [17]:
X_test = df_test.drop(columns = 'PassengerId', axis = 1)
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,5,0,0,7.8292,0
1,3,0,6,1,0,7.0000,1
2,2,1,7,0,0,9.6875,0
3,3,1,5,0,0,8.6625,1
4,3,0,4,1,1,12.2875,1
...,...,...,...,...,...,...,...
413,3,1,5,0,0,8.0500,1
414,1,0,5,0,0,108.9000,2
415,3,1,5,0,0,7.2500,1
416,3,1,5,0,0,8.0500,1


In [18]:
y = df_train['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [19]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()

clf.fit(X, y)


KNeighborsClassifier()

In [20]:
df_test['Survived'] =clf.predict(X_test)

In [21]:
df_test[['PassengerId', 'Survived']].to_csv(path_or_buf = '/Users/egmzvalerio/apps/iteso-data-analytics/data/raw/KNN_test.csv', index = False)